In [1]:
!pip install transformers -q
!pip install accelerate -q
!pip install bitsandbytes -q
!pip install peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.0 MB/s eta 0:00:00


In [2]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

In [3]:
import os
import json
import sys
import io
import random
import itertools
from typing import Any, Dict, List, Optional, Tuple, Union
import shutil
import logging
from dataclasses import dataclass, field

import torch
import torch.optim
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer, TrainerCallback
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config
from transformers import HfArgumentParser

import accelerate, bitsandbytes

from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType



In [4]:


# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", 40)


from google.colab import drive

drive.mount("/content/drive")

random_state = 42

dataset_path = "/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/all_dialogues_0.json"

def set_seed(seed=random_state):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    tf.random.set_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

Mounted at /content/drive


In [ ]:
pre_prompt = 'Тебя зовут Никита, тебе 21 год. Ты учишься на 3-ем курсе ВШЭ и занимаешься машинным обучением. Ты общаешься со своим другом, напиши ответ на его сообщение. '

In [ ]:

while True:
    print('-'*80)
    dialog = []
    while True:
        #print(dialog)
        msg = 'Собеседник: ' + input('Собеседник: ').strip()
        if len(msg) == 0:
            break

        dialog.append(msg)
        prompt = '<SC1>'+ pre_prompt +'Ваш прошлый диалог:' + '\n'.join(dialog) + ' Твой ответ:<extra_id_0>'
        input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
        print(f'PROMPT:{prompt}')
        print(dialog)
        out_ids = model.generate(input_ids=input_ids,
                                    max_length=50,
                                    eos_token_id=tokenizer.eos_token_id,
                                    early_stopping=True,
                                    do_sample=True,
                                    temperature=0.8,
                                    top_k=0,
                                    top_p=0.85)
        #dialog.pop(-1)
        t5_output = tokenizer.decode(out_ids[0][1:]).replace('<extra_id_0>','')
        if '</s>' in t5_output:
            t5_output = t5_output[:t5_output.find('</s>')].strip()

        print('Никита: {}'.format(t5_output))
        dialog.append('Никита: '+t5_output)

In [5]:
def load_samples(dataset_path, pre_prompt, tokenizer):
    samples = []
    with open(dataset_path, 'r', encoding="utf-8") as f:
        for sample in json.load(f):
            try:
                seed = '<SC1>' + pre_prompt +'Ваш прошлый диалог: '+ sample['context'] + ' Твой ответ: <extra_id_0>'
                reply = '<extra_id_0>' + sample['response']
                input_tokens = tokenizer.encode(seed, add_special_tokens=False, truncation=True, max_length=1024)
                output_tokens = tokenizer.encode(reply, add_special_tokens=False)  # , truncation=True, max_length=1024)
                if len(input_tokens) < 712 and len(output_tokens) < 712:
                    samples.append({'input_tokens': input_tokens,
                                    'output_tokens': output_tokens,
                                    'seed': seed,
                                    'reply': reply})
            except Exception as ex:
                print(ex)

    return samples


class FinetuneDataset(Dataset):
    def __init__(self, samples, tokenizer):
        self.tokenizer = tokenizer
        self.max_input_len = 0
        self.max_output_len = 0
        self.samples = []

        self.bos_token_id = tokenizer.encode('<s>', add_special_tokens=False)[0]
        self.eos_token_id = tokenizer.encode('</s>', add_special_tokens=False)[0]
        self.pad_token_id = tokenizer.encode('<pad>', add_special_tokens=False)[0]

        for sample in samples:
            input_ids = sample['input_tokens']
            output_ids = sample['output_tokens'] + [self.eos_token_id]
            self.samples.append((input_ids, output_ids))
            self.max_input_len = max(self.max_input_len, len(input_ids))
            self.max_output_len = max(self.max_output_len, len(output_ids))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index: int):
        input_ids, output_ids = self.samples[index]

        input_npad = self.max_input_len - len(input_ids)
        attention_mask = [1]*len(input_ids) + [0]*input_npad
        input_ids = input_ids + input_npad * [self.pad_token_id]

        output_npad = self.max_output_len - len(output_ids)
        labels = output_ids + output_npad * [-100]

        return {'input_ids': torch.LongTensor(input_ids),
                'attention_mask': attention_mask,
                'labels': torch.LongTensor(labels),
                }


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = transformers.GPT2Tokenizer.from_pretrained("ai-forever/FRED-T5-1.7B")
model = transformers.T5ForConditionalGeneration.from_pretrained("ai-forever/FRED-T5-1.7B", load_in_8bit=True, device_map="auto")


In [8]:
tokenizer.add_special_tokens({'bos_token': '<s>', 'eos_token': '</s>', 'pad_token': '<pad>'})

0

In [ ]:
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)


In [ ]:

peft_model = get_peft_model(prepare_model_for_int8_training(model), lora_config)
peft_model.print_trainable_parameters()

In [ ]:
import torch, gc

def empty_cache()
    gc.collect()
    torch.cuda.empty_cache()
    print('Done!')

In [9]:
train_samples = load_samples(dataset_path, tokenizer = tokenizer, pre_prompt = pre_prompt)
train_dataset = FinetuneDataset(train_samples, tokenizer)

In [10]:
len(train_dataset)

29308

In [ ]:



peft_training_args = TrainingArguments(
    fp16 = False,
    output_dir = '/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/models/t5/out',
    overwrite_output_dir=False,
    optim =  "adafactor",
    learning_rate  = 1e-3,
    weight_decay=0,
    lr_scheduler_type  = 'constant',
    num_train_epochs = 1,
    per_device_train_batch_size = 6,
    gradient_accumulation_steps  = 8, #8
    gradient_checkpointing=False,
    report_to  = 'tensorboard',
    logging_strategy  = 'steps',
    logging_dir='/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/models/t5/logs',
    logging_steps  = 1,
    save_steps=10,
    save_total_limit=None,
    seed=42,

)

#    weight_decay=0,

In [ ]:
trainer = Trainer(
        model=peft_model,
        args=peft_training_args,
        train_dataset=train_dataset,
        tokenizer=tokenizer,
        data_collator=None,

    )

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32) #float32

In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,4.356000
2,4.432100
3,4.279300
4,4.184200
5,4.095700
6,3.722200
7,3.980700
8,3.674100
9,3.617000
10,3.498700


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

In [11]:
from peft import PeftModel, PeftConfig

checkpoint = '/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/models/t5/out/checkpoint-140'
# peft_model = PeftModel.from_pretrained(prepare_model_for_int8_training(model), checkpoint)
#peft_model = PeftModel.from_pretrained(model, checkpoint, device_map="auto", is_trainable=True)

#peft_model = prepare_model_for_int8_training(peft_model)
#peft_model = peft_model.merge_and_unload()

In [12]:
if checkpoint is not None:
    peft_config = PeftConfig.from_pretrained(checkpoint)
    lora_rank = peft_config.r
    lora_alpha = peft_config.lora_alpha
    lora_dropout = peft_config.lora_dropout


peft_config = LoraConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,
        bias="none",
        target_modules=["q", "v"],
        r=lora_rank,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
    )
peft_model = get_peft_model(prepare_model_for_int8_training(model), peft_config)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [13]:

peft_training_args = TrainingArguments(
    fp16 = False,
    output_dir = '/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/models/t5/out',
    overwrite_output_dir=False,
    optim =  "adafactor",
    learning_rate  = 1e-3,
    weight_decay=0,
    lr_scheduler_type  = 'constant',
    num_train_epochs = 1,
    per_device_train_batch_size = 6,
    gradient_accumulation_steps  = 8, #8
    gradient_checkpointing=False,
    report_to  = 'tensorboard',
    logging_strategy  = 'steps',
    logging_dir='/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/models/t5/logs',
    logging_steps  = 1,
    save_steps=20,
    save_total_limit=None,
    seed=42,

)






trainer = Trainer(
        model=peft_model,
        args=peft_training_args,
        train_dataset=train_dataset,
        tokenizer=tokenizer,
        data_collator=None,

    )



for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32) #float32


In [ ]:
trainer.train(resume_from_checkpoint=checkpoint)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
141,3.086000
142,3.002900
143,2.891100
144,2.928700
145,2.970300
146,3.100700
147,2.955800
148,3.101500
149,3.068800
150,2.960800


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [ ]:
trainer.model == peft_model

True

In [ ]:
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

***** train metrics *****
  epoch                    =        1.0
  total_flos               = 49374623GF
  train_loss               =     3.6083
  train_runtime            = 3:21:02.64
  train_samples_per_second =      0.663
  train_steps_per_second   =      0.014


In [ ]:
peft_model_id="/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/models/t5/results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)


# trainer.save_model(output_dir=training_args.output_dir)
# tokenizer.save_pretrained(training_args.output_dir)
# model.save_pretrained(training_args.output_dir)

('/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/models/t5/results/tokenizer_config.json',
 '/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/models/t5/results/special_tokens_map.json',
 '/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/models/t5/results/vocab.json',
 '/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/models/t5/results/merges.txt',
 '/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/models/t5/results/added_tokens.json')

In [ ]:
peft_model_id

'results'

In [9]:
peft_model_id = '/content/drive/MyDrive/PROJECTS_COMPETITIONS/digital_copy/models/t5/out/checkpoint-140'

In [10]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
#peft_model_id = "results"
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
#model = transformers.T5ForConditionalGeneration.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map="auto")
#tokenizer = transformers.GPT2Tokenizer.from_pretrained(config.base_model_name_or_path)


#t5_tokenizer = transformers.GPT2Tokenizer.from_pretrained("Den4ikAI/FRED-T5-XL-chitchat")
#t5_model = transformers.T5ForConditionalGeneration.from_pretrained("Den4ikAI/FRED-T5-XL-chitchat")

# Load the Lora model
peft_trained_model = PeftModel.from_pretrained(model, peft_model_id, device_map="auto", torch_dtype=torch.float16)
peft_trained_model.eval()


print("Peft model loaded")

Peft model loaded


In [ ]:
#   seed = '<SC1>' + pre_prompt +'Ваш прошлый диалог: '+ sample['context'] + ' Твой ответ:<extra_id_0>'
#                 reply = '<extra_id_0>' + sample['response']

In [ ]:
from torch import autocast

In [ ]:
peft_trained_model.__dict__

In [16]:

while True:
    print('-'*80)
    dialog = []
    while True:
        #print(dialog)
        msg = 'Собеседник: ' + input('Собеседник: ').strip()
        if len(msg) == 0:
            break

        dialog.append(msg)
        prompt = '<SC1>'+ pre_prompt +'Ваш прошлый диалог:' + '\n'.join(dialog) + ' Твой ответ:<extra_id_0>'
        input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
        print(f'PROMPT:{prompt}')
        print(dialog)
        # with autocast('cuda'):
        out_ids = peft_trained_model.generate(input_ids=input_ids,
                                    max_length=50,
                                    eos_token_id=tokenizer.eos_token_id,
                                    early_stopping=True,
                                    do_sample=True,
                                    temperature=0.8,
                                    top_k=50,
                                    top_p=0.85)
        #dialog.pop(-1)
        t5_output = tokenizer.decode(out_ids[0][1:]).replace('<extra_id_0>','')
        if '</s>' in t5_output:
            t5_output = t5_output[:t5_output.find('</s>')].strip()

        print('Никита: {}'.format(t5_output))
        dialog.append('Никита: '+t5_output)

--------------------------------------------------------------------------------
Собеседник: Кто для тебя самай любимы человек на земле?
PROMPT:<SC1>Тебя зовут Никита, тебе 21 год. Ты учишься на 3-ем курсе ВШЭ и занимаешься машинным обучением. Ты общаешься со своим другом, напиши ответ на его сообщение. Ваш прошлый диалог:Собеседник: Кто для тебя самай любимы человек на земле? Твой ответ:<extra_id_0>
['Собеседник: Кто для тебя самай любимы человек на земле?']
Никита: я не знаю, может быть, человек, который в данный момент находится рядом со мной 
какой-нибудь близкий друг или подруга 
или любимая девушка
Собеседник: прекрасно я считаю 
PROMPT:<SC1>Тебя зовут Никита, тебе 21 год. Ты учишься на 3-ем курсе ВШЭ и занимаешься машинным обучением. Ты общаешься со своим другом, напиши ответ на его сообщение. Ваш прошлый диалог:Собеседник: Кто для тебя самай любимы человек на земле?
Никита: я не знаю, может быть, человек, который в данный момент находится рядом со мной 
какой-нибудь близкий дру

KeyboardInterrupt: ignored